In [1]:
from IPython.display import display
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
       .master("local[*]")\
       .appName("SparklingWaterApp")\
       .getOrCreate()

24/04/13 11:27:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
# pip3 install h2o_pysparkling_3.3

In [5]:
from pysparkling import *
import h2o
conf = H2OConf().setLogLevel("WARN")
hc = H2OContext.getOrCreate(conf)

24/04/13 11:27:43 WARN InternalH2OBackend: Increasing 'spark.locality.wait' to value 0 (Infinitive) as we need to ensure we run on the nodes with H2O
04-13 11:27:45.109 172.17.0.2:54321      3117     Thread-4  WARN water.default: SECURITY_WARNING: web_ip is not specified. H2O Rest API is listening on all available interfaces.
04-13 11:27:45.112 172.17.0.2:54321      3117     Thread-4  WARN water.default: Flatfile configuration does not include self: /172.17.0.2:54321, but contains []
Connecting to H2O server at http://623587162df7:54323 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,30 days
H2O_cluster_name:,sparkling-water-NBuser_local-1713007659000
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.46.0.1-1-3.3
 * H2O name: sparkling-water-NBuser_local-1713007659000
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,172.17.0.2,54321)
  ------------------------

  Open H2O Flow in browser: http://623587162df7:54323 (CMD + click in Mac OSX)

    


In [6]:
frame = h2o.import_file("loan.csv")
sparkDF = hc.asSparkFrame(frame).drop("addr_state")
display(sparkDF.limit(10).toPandas())

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36 months,10.65,10,RENT,24000.0,credit_card,27.65,0,83.7,9,0,26,verified
1,2500,60 months,15.27,0,RENT,30000.0,car,1.00,0,9.4,4,1,12,verified
2,2400,36 months,15.96,10,RENT,12252.0,small_business,8.72,0,98.5,10,0,10,not verified
3,10000,36 months,13.49,10,RENT,49200.0,other,20.00,0,21.0,37,0,15,verified
4,5000,36 months,7.90,3,RENT,36000.0,wedding,11.20,0,28.3,12,0,7,verified
5,3000,36 months,18.64,9,RENT,48000.0,car,5.35,0,87.5,4,0,4,verified
6,5600,60 months,21.28,4,OWN,40000.0,small_business,5.55,0,32.6,13,1,7,verified
7,5375,60 months,12.69,0,RENT,15000.0,other,18.08,0,36.5,3,1,7,verified
8,6500,60 months,14.65,5,OWN,72000.0,debt_consolidation,16.12,0,20.6,23,0,13,not verified
9,12000,36 months,12.69,10,OWN,75000.0,debt_consolidation,10.78,0,67.1,34,0,22,verified


In [7]:
[trainingDF, testingDF] = sparkDF.randomSplit([0.7, 0.3])

print(trainingDF.count())
print(testingDF.count())

708
291


In [8]:
from pysparkling.ml import H2OGLM

predictors = ["loan_amnt", 
              "emp_length", 
              "annual_inc", 
              "dti",
              "revol_util",
              "total_acc",
              "longest_credit_length"]

target = "bad_loan"

glm_estimator = H2OGLM(family = "binomial",
     labelCol = target,
     featuresCols = predictors,                       
     computePValues = True)

model = glm_estimator.fit(trainingDF)

|██████████████████████████████████████████████████| 100%
Model Details
H2OGLM
Model Key: GLM_f0f10ecebc05

Model summary
Family: binomial
Link: logit
Regularization: Elastic Net (alpha = 0.5, lambda = 7.537E-5 )
Number of Predictors Total: 7
Number of Active Predictors: 7
Number of Iterations: 3
Training Frame: frame_rdd_191771543862

Training metrics
PRAUC: 0.2982232969532082
Nobs: 708.0
Logloss: 0.4944778413265193
Gini: 0.22589575549702956
RMSE: 0.39815594733982834
ResidualDeviance: 700.1806233183513
NullDeviance: 717.8951900830491
ScoringTime: 1.713007699492E12
Loglikelihood: 0.0
MSE: 0.15852815840207618
R2: 0.026588365367081424
NullDegreesOfFreedom: 707.0
MeanPerClassError: 0.4156979236846941
AUC: 0.6129478777485148
AIC: 716.1806233183513
ResidualDegreesOfFreedom: 700.0

More info available using methods like:
getFeatureImportances(), getScoringHistory(), getCrossValidationScoringHistory()


In [9]:
display(model
        .getCoefficients()\
        .toPandas())

,names,Coefficients,Std. Error,z value,p value,Standardized Coefficients
0,Intercept,-2.174779,0.466463,-4.662273,0.000003,-1.408689
1,loan_amnt,0.000029,0.000015,1.916654,0.055282,0.206600
2,emp_length,-0.001917,0.028225,-0.067934,0.945838,-0.006593
3,annual_inc,-0.000009,0.000004,-2.219874,0.026427,-0.305940
4,dti,0.018976,0.017213,1.102457,0.270263,0.118157
5,revol_util,0.009227,0.004567,2.020602,0.043321,0.211660
6,total_acc,0.000379,0.011493,0.032975,0.973695,0.003826
7,longest_credit_length,0.010295,0.017362,0.592976,0.553197,0.061724


In [10]:
display(model\
        .getFeatureImportances()\
        .toPandas())

,Variable,Relative Importance,Scaled Importance,Percentage
0,annual_inc,0.305940,1.000000,0.334544
1,revol_util,0.211660,0.691833,0.231448
2,loan_amnt,0.206600,0.675296,0.225916
3,dti,0.118157,0.386209,0.129204
4,longest_credit_length,0.061724,0.201752,0.067495
5,emp_length,0.006593,0.021551,0.007210
6,total_acc,0.003826,0.012505,0.004183


In [11]:
display(model\
        .transform(testingDF)\
        .crosstab(target, "prediction")\
        .toPandas())

,bad_loan_prediction,0,1
0,1,15,34
1,0,104,138


In [12]:
from ai.h2o.sparkling.ml.algos.classification import H2OXGBoostClassifier

target = "bad_loan"

min_rows = int(trainingDF.count() * 0.05)

xgb_classifier = H2OXGBoostClassifier(labelCol = target, 
                                 booster = "gbtree",
                                 ntrees = 250, 
                                 minRows = min_rows,
                                 detailedPredictionCol = "prediction")

model = xgb_classifier.fit(trainingDF)

[11:28:38] WARNING: /dot/src/learner.cc:248: No visible GPU is found, setting `gpu_id` to -1


|██████████████████████████████████████████████████| 100%
Model Details
H2OXGBoost
Model Key: XGBoost_135da5adf1d9

Model summary
Number of Trees: 250

Training metrics
PRAUC: 0.5079810724042972
Nobs: 708.0
Logloss: 0.42870121567879504
Gini: 0.5627978195626875
RMSE: 0.3686557211692449
ScoringTime: 1.713007724641E12
Loglikelihood: NaN
MSE: 0.13590704075081603
R2: 0.16548898297400583
MeanPerClassError: 0.27182581000796224
AUC: 0.7813989097813437
AIC: NaN

More info available using methods like:
getFeatureImportances(), getScoringHistory(), getCrossValidationScoringHistory()


In [13]:
display(model\
        .getFeatureImportances()\
        .toPandas())

,Variable,Relative Importance,Scaled Importance,Percentage
0,int_rate,65.101776,1.000000,0.321005
1,dti,30.525665,0.468891,0.150517
2,revol_util,28.039978,0.430710,0.138260
3,annual_inc,22.346027,0.343248,0.110184
4,emp_length,14.745679,0.226502,0.072708
5,loan_amnt,12.612086,0.193729,0.062188
6,longest_credit_length,11.538143,0.177232,0.056892
7,total_acc,7.905815,0.121438,0.038982
8,home_ownership.RENT,5.445649,0.083648,0.026852
9,purpose.credit_card,4.545235,0.069817,0.022412


In [14]:
display(model\
        .transform(testingDF)\
        .crosstab(target, "prediction")\
        .toPandas())

04-13 11:28:53.681 172.17.0.2:54321      3117     Thread-4  WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,bad_loan_prediction,0,1
0,1,29,20
1,0,181,61


In [15]:
from pysparkling.ml import H2ODeepLearning

target = "bad_loan"

cat_columns = ["term", "home_ownership", "purpose", "verification_status"]

num_columns = ["loan_amnt", 
              "emp_length", 
              "annual_inc", 
              "dti",
              "revol_util",
              "total_acc",
              "longest_credit_length"]

features = cat_columns + num_columns

estimator = H2ODeepLearning(
                distribution = "bernoulli",
                hidden = [100, 50, 20],
                epochs = 1000,
                columnsToCategorical = cat_columns,
                featuresCols = features,
                labelCol = target,
                detailedPredictionCol = "prediction")

model = estimator.fit(trainingDF)

|██████████████████████████████████████████████████| 100%
Model Details
H2ODeepLearning
Model Key: DeepLearning_783419e8fe4b

Model summary
Layer: 1
Units: 31
Type: Input
Dropout: 0.0
L1: null
L2: null
Mean Rate: null
Rate RMS: null
Momentum: null
Mean Weight: null
Weight RMS: null
Mean Bias: null
Bias RMS: null

Layer: 2
Units: 100
Type: Rectifier
Dropout: 0.0
L1: 0.0
L2: 0.0
Mean Rate: 0.6698042243970168
Rate RMS: 0.4062594175338745
Momentum: 0.0
Mean Weight: -0.009511405851817471
Weight RMS: 0.17161691188812256
Mean Bias: 0.4556717675846356
Bias RMS: 0.10338079929351807

Layer: 3
Units: 50
Type: Rectifier
Dropout: 0.0
L1: 0.0
L2: 0.0
Mean Rate: 0.8883474355719984
Rate RMS: 0.2442569136619568
Momentum: 0.0
Mean Weight: -0.010040520213573472
Weight RMS: 0.16858923435211182
Mean Bias: 0.9857253142808254
Bias RMS: 0.07146322727203369

Layer: 4
Units: 20
Type: Rectifier
Dropout: 0.0
L1: 0.0
L2: 0.0
Mean Rate: 0.8668869805929135
Rate RMS: 0.2774808406829834
Momentum: 0.0
Mean Weight: 0.00

In [16]:
display(model\
        .getFeatureImportances()\
        .toPandas())

,Variable,Relative Importance,Scaled Importance,Percentage
0,annual_inc,1.000000,1.000000,0.050926
1,loan_amnt,0.951819,0.951819,0.048473
2,longest_credit_length,0.947105,0.947105,0.048233
3,emp_length,0.921309,0.921309,0.046919
4,dti,0.882116,0.882116,0.044923
5,revol_util,0.850610,0.850610,0.043318
6,purpose.vacation,0.828945,0.828945,0.042215
7,total_acc,0.825610,0.825610,0.042045
8,verification_status.not verified,0.750574,0.750574,0.038224
9,verification_status.verified,0.735123,0.735123,0.037437


In [17]:
display(model\
        .transform(testingDF)\
        .crosstab(target, "prediction")\
        .toPandas())

,bad_loan_prediction,0,1
0,0,218,24
1,1,41,8


In [18]:
spark.stop()